In [1]:
import numpy as np, pandas as pd


In [2]:
def pairwiseDist(x, y=None):
    if y is None:
        y = x
    return np.sum((x[:,None]-y)**2,axis=2)**0.5

In [ ]:
def mostCommonElem(elems):
    unq, unqCnt = np.unique(elems, return_counts=True)
    maxIdx = np.where(unqCnt==np.max(unqCnt))
    if len(maxIdx) > 1: # there's a tie

################################################################################
def KNN(trainX, trainY, testX, K, regression=False):
    dists = pairwiseDist(testX,trainX) # all pairwise dist of two datasets
    kIdx = np.argpartition(dists, K)[:,:K] # find idx of smallest K elements
    neighbs = trainY[kIdx]
    
    testY = np.empty(testX.shape, trainY.dtype) # pre-allocate test data labels
    if regression:
        pass
    else:
        for n,knn in enumerate(neighbs):
            testY[n] = mostCommonElem(knn)
    
    

In [42]:
np.empty(testX.shape, dtype=int)

array([[1208407136,        146, 1208407328,        146, 1208407184],
       [       146,          0,          0,          0,          0],
       [         0,          0,          0,          0, 1208406512],
       [       146,          0,          0,          0,          0],
       [         0,          0, 1208407424,        146,          0],
       [         0,          0,          0, 1208407040,        146],
       [1208407808,        146, 1208407472,        146,          0],
       [         0,          0,          0, 1208407856,        146],
       [         0,          0,          0,          0, 1208406320],
       [       146,          0,          0,          0,          0],
       [1208407760,        146, 1208407376,        146,          0],
       [         0,          0,          0, 1208406848,        146],
       [         0,          0,          0,          0,          0]])

In [20]:
testX = np.vstack([np.random.randn(8,5)+9, np.random.randn(5,5)+2])
testY = np.array(['B']*8 + ['S']*5)
testPred = np.vstack([np.random.randn(2,5)+9, np.random.randn(2,5)+2])
d = pairwiseDist(testPred, testX)

In [21]:
d

array([[ 1.38359908,  2.76655379,  1.84680373,  2.11989089,  1.58085558,
         2.2267154 ,  1.84234042,  1.76268907, 16.64342548, 16.09662031,
        14.6928764 , 15.04998073, 15.81889807],
       [ 1.59203808,  1.73620462,  1.52847415,  1.91510187,  1.72709241,
         1.24535541,  2.00042648,  1.83680441, 16.34601047, 15.5750117 ,
        14.14526648, 14.6808198 , 15.29559435],
       [17.15466327, 16.57381648, 16.26667696, 16.41192156, 17.53446897,
        16.94620681, 16.53636849, 16.02875769,  2.35041518,  2.28435493,
         3.06293502,  2.5858431 ,  2.60257134],
       [18.16993491, 17.92905315, 17.47774017, 17.57867994, 18.58093978,
        18.28943574, 17.66175795, 17.31161639,  2.35273622,  4.55587878,
         4.79249561,  3.59293427,  3.9704747 ]])

In [39]:
z = np.argpartition(d, 3)[:,:3]
z

array([[ 4,  0,  7],
       [ 5,  2,  0],
       [ 8,  9, 11],
       [12, 11,  8]], dtype=int64)

In [40]:
testY[z]

array([['B', 'B', 'B'],
       ['B', 'B', 'B'],
       ['S', 'S', 'S'],
       ['S', 'S', 'S']], dtype='<U1')